<a href="https://colab.research.google.com/github/anutom20/Amazon_web_scraping/blob/main/amazon_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

  Start The Webdriver
  

In [3]:
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [4]:
# open it, go to the website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
wd.get(url)

Extract the collection

In [5]:
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [6]:
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

In [7]:
len(results)

16

Prototype the Record

In [54]:
item = results[1]


In [55]:
atag = item.h2.a

In [56]:
product_url = 'https://www.amazon.in' + atag.get('href')

In [57]:
price_parent = item.find('span','a-price')

In [58]:
product_price = price_parent.find('span' , 'a-offscreen').text[1:]

In [59]:
product_rating = item.i.text

In [60]:
product_name = item.h2.a.span.text

In [61]:
no_of_reviews = item.find('span',{'class': 'a-size-base'}).text

PART-2 Get  Description , ASIN ,   Manufacturer & Product Description form the url for a single product

In [62]:
# get single product information from the scraped url
wd.get(product_url)
single_prod_soup = BeautifulSoup(wd.page_source, 'html.parser')


In [63]:
#get single product description

product_desc = single_prod_soup.find('div', {'id' : 'feature-bullets'}).find('ul')

single_product_desc = ''
for li in product_desc.find_all('li'):
  single_product_desc += li.text
print(single_product_desc)    

 Combination of functional & safety features in stylish design, Soft mesh back with 8 mm foam padded 2 Main Compartment, 1 Slip In Pocket inside the bag, Printed Design, Mesh bottle holder on the side,  


In [64]:
# get ASIN
product_asin = product_url.split("/dp/")[1].split("/ref")[0]

In [65]:
#get product manufacturer
manufacturer_text = single_prod_soup.find('a', {'id' : 'bylineInfo'}).text

manufacturer = ''

if(manufacturer_text.find('Visit') != -1 and manufacturer_text.find('Store') != -1):
  manufacturer = manufacturer_text.split('the')[1].split('Store')[0]
else:
  manufacturer = manufacturer_text.split(':')[1]
print(manufacturer)


 Skybags 


In [66]:
#get product description
prod_desc = single_prod_soup.find('div', {'id' : 'productDescription'})

if prod_desc and prod_desc.p.text:
  prod_desc = prod_desc.p.text
else:
  prod_desc = single_product_desc

print(prod_desc)


 Ideal for a college student who does not carry a lot of books or laptop to college. Very stylish, slim and feature loaded Backpack 


Generalize the pattern

In [21]:
def extract_single_item(item):
  # extract and return data from a single item

  #Product Url
  atag = item.h2.a
  product_url = 'https://www.amazon.in' + atag.get('href')

  #Product Name
  product_name = item.h2.a.span.text
  #Product Price
  try:
    price_parent = item.find('span','a-price')
    product_price = price_parent.find('span' , 'a-offscreen').text[1:]
  except AttributeError:
    return
    
  try:
    #Product Rating
    product_rating = item.i.text

    #No of reviews
    no_of_reviews = item.find('span',{'class': 'a-size-base'}).text
  except AttributeError:
    product_rating = ''
    no_of_reviews = ''


  # part-2 , getting additional product info
  wd.get(product_url)
  single_product_soup = BeautifulSoup(wd.page_source, 'html.parser')

    
  #description

  product_desc = single_product_soup.find('div', {'id' : 'feature-bullets'}).find('ul')

  single_product_desc = ''
  if product_desc:
    for li in product_desc.find_all('li'):
      single_product_desc += li.text  
  else:
    single_product_desc = 'Not available'

    
  # get ASIN
  product_asin = ''
  try:
    product_asin = product_url.split("/dp/")[1].split("/ref")[0]
  except:
    product_asin = 'n/a'
  #product manufacturer
  try:
    manufacturer_text = single_product_soup.find('a', {'id' : 'bylineInfo'}).text
  except AttributeError:
    manufacturer_text = ''

  manufacturer = ''

  if manufacturer_text:

    if(manufacturer_text.find('Visit') != -1 and manufacturer_text.find('Store') != -1):
      manufacturer = manufacturer_text.split('the')[1].split('Store')[0]
    else:
      manufacturer = manufacturer_text.split(':')[1]

  #product description
  prod_desc = single_product_soup.find('div', {'id' : 'productDescription'})

  if prod_desc and prod_desc.p:
    prod_desc = prod_desc.p.text
  else:
    prod_desc = single_product_desc
  
  




  result = (product_url , product_name , product_price , product_rating , no_of_reviews, single_product_desc , product_asin , prod_desc , manufacturer)

  return result








Store the products in a list

In [22]:
records = []
items = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in items:
  record = (extract_single_item(item))
  if record:
    records.append(record)

In [41]:
records[1]

('https://www.amazon.in/Skybags-Brat-Black-Casual-Backpack/dp/B08Z1HHHTD/ref=sr_1_2?crid=2M096C61O4MLT&keywords=bags&qid=1668884721&qu=eyJxc2MiOiI4LjA0IiwicXNhIjoiOC4wNiIsInFzcCI6IjYuNzgifQ%3D%3D&sprefix=ba%2Caps%2C283&sr=8-2',
 'Skybags Brat Black 46 Cms Casual Backpack',
 '669',
 '4.1 out of 5 stars',
 '2,558',
 ' Combination of functional & safety features in stylish design, Soft mesh back with 8 mm foam padded 2 Main Compartment, 1 Slip In Pocket inside the bag, Printed Design, Mesh bottle holder on the side,  ',
 'B08Z1HHHTD',
 ' Ideal for a college student who does not carry a lot of books or laptop to college. Very stylish, slim and feature loaded Backpack ',
 ' Skybags ')

Getting next page

In [24]:
def get_url(page_no):
  template  = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

  # add page no to the url
  template += '&page={}'
  url = template.format(str(page_no))
  return url



Getting Complete Data From All The 20 Pages

In [25]:
def get_complete_data():
  records = []
  for page in range (1,21):
    url = get_url(page)
    wd.get(url)
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    items = soup.find_all('div', {'data-component-type' : 's-search-result'})

    for item in items:
      record = (extract_single_item(item))
      if record:
        records.append(record)
  return records      
       



Getting Final Result

In [26]:
final_res = get_complete_data()
len(final_res)


329

In [27]:
final_res[0:5]

[('https://www.amazon.in/Half-Moon-Waterproof-Backpack-Students/dp/B085MHDJ93/ref=sr_1_1?crid=2M096C61O4MLT&keywords=bags&qid=1668884793&qu=eyJxc2MiOiI4LjA0IiwicXNhIjoiOC4wNiIsInFzcCI6IjYuNzgifQ%3D%3D&sprefix=ba%2Caps%2C283&sr=8-1',
  'Half Moon 35 L Casual Waterproof Laptop Bag/Backpack for Men Women Boys Girls/Office School College Teens & Students with Rain Cover (18 Inch) (Navy)',
  '699',
  '3.9 out of 5 stars',
  '13,861',
  ' SPACIOUS AND LIGHTWEIGHT: Light weight and spacious laptop backpack with 3 compartments. One compartment is for carrying laptop inside (Upto 17 inches), with padding for safety, and space for laptop accessories like charger, etc. Second large compartment for books, files and all your day to day essentials. And third compartment is for your extra stuff. It has provision for holding bottles on both sides and has padded shoulder straps for your comfort   COMFORT WITH QUALITY: This casual bag has Wide, Padded and Adjustable shoulder straps and Padded Back for c

Writing the results to the CSV file

In [29]:
# save data to a csv file

with open ('amazon_scraping_results.csv', 'w' , newline='', encoding='utf-8') as f:
  writer = csv.writer(f)
  writer.writerow(['ProductUrl','ProductName', 'Price', 'Rating', 'NoOfReviews', 'Description', 'ASIN', 'Product Description', 'Manufacturer'])
  writer.writerows(final_res)